In [1]:
import tensorflow as tf
import pandas as pd
import cv2
import os

# GPU memory limit
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Import own modules
import lib.custom_callbacks as callbacks
import lib.PrepareDataset as dt
import lib.constants as ctes
import lib.RCAN as RCAN

######################################################################################
# DEFINE ARGUMENTS

# Load datasets from CSV files
TRAINING_DATASETS = [pd.read_csv('data/0_csvs/DIV2K_train_HR.csv')]
VALIDATION_DATASETS = [pd.read_csv('data/0_csvs/DIV2K_valid_HR.csv')]

# Define dataset parameters
TRAINING_COLOR_MODE = ctes.COLOR_MODE.RGB
TRAINING_SCALE = 3
TRAINING_BATCH_SIZE = 16
TRAINING_PATCH_SIZE = 48
TRAINING_DATA_AUG = True
TRAINING_SHUFFLE = True
TRAINING_REPEAT = 200 # ----------------- Según la implementación en TF 1.13, se valida cada 10.000 training steps. Al ser 800 imágenes de entrenamiento y un batch de 16, hay 50 steps por época
                      #                   Hay que realizar 10.000 iteraciones antes de validar. Estas son equivalentes a 200 épocas de entrenamiento sin validación, osea, repetir 200 veces el dataset
TRAINING_NORMALIZE = False

VALIDATION_COLOR_MODE = TRAINING_COLOR_MODE
VALIDATION_SCALE = TRAINING_SCALE
VALIDATION_BATCH_SIZE = 1
VALIDATION_PATCH_SIZE = None
VALIDATION_DATA_AUG = False
VALIDATION_SHUFFLE = False
VALIDATION_REPEAT = None
VALIDATION_NORMALIZE = False

FILENAME = 'nb04_train_rcan-scale3_replica_original'

# Get model info from file name
info_from_script_name = FILENAME.split('_') 

model_structure_name = info_from_script_name[2].upper() 

model_name = info_from_script_name[3:]            
model_name = ('_'.join(['model'] + model_name)).split('.')[0]
model_name_for_metrics = model_structure_name + '_' + model_name         

BASE_MODEL_PATH = 'TRAINED_MODELS_NEW/'
BASE_METRIC_PATH = 'TRAINING_METRIC_EVOLUTIONS_NEW/'

if not os.path.exists(BASE_MODEL_PATH):
    os.mkdir(BASE_MODEL_PATH)
if not os.path.exists(BASE_METRIC_PATH):
    os.mkdir(BASE_METRIC_PATH)

save_path = BASE_MODEL_PATH + model_structure_name + '/'
save_metrics_path = BASE_METRIC_PATH

if not os.path.exists(save_path):
    os.mkdir(save_path)

# Load datasets as PrepareDataset objects
train = dt.PrepareDataset(dataframes=TRAINING_DATASETS, 
                          channel_mode=TRAINING_COLOR_MODE, 
                          scale=TRAINING_SCALE,
                          batch_size=TRAINING_BATCH_SIZE, 
                          patch_size=TRAINING_PATCH_SIZE,
                          data_augmentation=TRAINING_DATA_AUG, 
                          shuffle=TRAINING_SHUFFLE, 
                          repeat=TRAINING_REPEAT,
                          normalize=TRAINING_NORMALIZE)

val = dt.PrepareDataset(dataframes=VALIDATION_DATASETS, 
                        channel_mode=VALIDATION_COLOR_MODE, 
                        scale=VALIDATION_SCALE, 
                        batch_size=VALIDATION_BATCH_SIZE,
                        patch_size=VALIDATION_PATCH_SIZE, 
                        data_augmentation=VALIDATION_DATA_AUG,
                        shuffle=VALIDATION_SHUFFLE,
                        repeat=VALIDATION_REPEAT,
                        normalize=VALIDATION_NORMALIZE)

print('----------------------------------')
print('Num images for training:', sum(x.shape[0] for x in TRAINING_DATASETS))
print('Num images for validating:', sum(x.shape[0] for x in VALIDATION_DATASETS))
print('----------------------------------')

def aux_fn(datasets, condition_fn):
    return condition_fn(condition_fn(cv2.imread(row.path).shape[1:-1]) for i in range(len(datasets)) for row in datasets[i].itertuples())

print('Smallest image in train dataset:', aux_fn(TRAINING_DATASETS, min))
smallest_validation_size = aux_fn(VALIDATION_DATASETS, min)
print('Smallest image in validation dataset:', smallest_validation_size)
print('----------------------------------')
print('Biggest image in train dataset:', aux_fn(TRAINING_DATASETS, max))
biggest_validation_image = aux_fn(VALIDATION_DATASETS, max)
print('Biggest image in validation dataset:', biggest_validation_image)
print('----------------------------------')

2021-10-14 21:16:34.913986: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-10-14 21:16:35.861367: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-14 21:16:35.861947: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-14 21:16:35.895787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-14 21:16:35.896009: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 119.24GiB/s
2021-10-14 21:16:35.896023: I tensorflow/stream_executor/platform/def

1 Physical GPUs, 1 Logical GPUs
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
----------------------------------
Num images for training: 800
Num images for validating: 100
----------------------------------
Smallest image in train dataset: 1116
Smallest image in validation dataset: 1356
----------------------------------
Biggest image in train dataset: 2040
Biggest image in validation dataset: 2040
----------------------------------


In [2]:
# Define model
NUM_RESIDUAL_GROUPS = 10
NUM_RESIDUAL_BLOCKS = 20
NUM_FEATURES = 64
KERNEL_SIZE = 3
REDUCTION = 16
NUM_IMAGE_CHANNELS = TRAINING_COLOR_MODE # 3
SCALE = TRAINING_SCALE
NORMALIZATION = False

model = RCAN.get_RCAN(NUM_RESIDUAL_GROUPS,
                        NUM_RESIDUAL_BLOCKS,
                        NUM_FEATURES,
                        KERNEL_SIZE,
                        REDUCTION,
                        NUM_IMAGE_CHANNELS,
                        SCALE,
                        NORMALIZATION)

# model.summary()
types = {
    'Conv': tf.keras.layers.Conv2D,
    'ReLU': tf.keras.layers.ReLU,
    'Add': tf.keras.layers.Add 
}

total_num = 0

for k in types:
    num = len(list(filter(lambda x: type(x) == types[k], model.layers)))
    print('   Num', k, 'layers:', num)
    total_num += num
print('Total layers:', total_num)

print('Num layers:', len(model.layers))
print('Num parameters:', model.count_params())

print('Num trainable variables:', sum(len(l.trainable_variables) for l in model.layers))

# Save model structure to JSON file
if os.path.exists(save_path + model_name + '.json'):
    os.remove(save_path + model_name + '.json')
with open(save_path + model_name + '.json', 'w') as json_file:
    json_file.write(model.to_json())

   Num Conv layers: 814
   Num ReLU layers: 200
   Num Add layers: 211
Total layers: 1225
Num layers: 1631
Num parameters: 15629283
Num trainable variables: 1628


In [ ]:
# Metrics for model evaluation
METRICS = [
    ctes.METRIC_FUNCTIONS.PSNR,
    ctes.METRIC_FUNCTIONS.SSIM,
    ctes.METRIC_FUNCTIONS.SSIM_MS, # ---- https://github.com/tensorflow/tensorflow/issues/33840
                                   #      MODIFIED ARGUMENTS IN SSIM MULTISCALE (lib.custom_metric_functions.py)
    ctes.METRIC_FUNCTIONS.MSE,
    ctes.METRIC_FUNCTIONS.MAE,
    ctes.METRIC_FUNCTIONS.SOBEL
]

# Metrics for saving model's checkpoints
METRICS_CHECKPOINTS = [
    (ctes.METRICS_ALL.PSNR, ctes.METRICS_ALL.VAL_PSNR, 'max'),
    (ctes.METRICS_ALL.SSIM, ctes.METRICS_ALL.VAL_SSIM, 'max'),
    # (ctes.METRICS_ALL.SOBEL, ctes.METRICS_ALL.VAL_SOBEL, 'min')
]

# Define training hyperparameters
INITIAL_LEARNING_RATE = 0.0001

NUM_EPOCHS = 100 # Nº máximo de iteraciones de entrenamiento: 1.000.000 -- Como antes definimos una época como 10.000 iteraciones (repetir 200 veces el dataset de 800 imágenes dividio en batches de 16)
                 # 1.000.000 de iteraciones serán 100 épocas

OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=INITIAL_LEARNING_RATE)
LOSS_FUNCTION = train.get_loss_function(ctes.LOSS_FUNCTIONS.MAE)

# Define callbacks

# Se indica en el paper original, y se aplica en la implementación en TF 1.13, que cada 200.000 iteraciones, el learning rate se debe reducir a la mitad
# Estableciendo que una época se compone de 200 repeticiones del dataset de entrenamiento para conseguir 10.000 iteraciones, cada 20 'épocas' se deberá reducir a la mitad el learning rate

learning_rate_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(callbacks.create_scheduler_function(20, 0.5)) # Cada 20 épocas, multiplicar lr por 0.5

# Model checkpoints callbacks
checkpoint_callbacks = [tf.keras.callbacks.ModelCheckpoint(save_path + model_name + '_best_' + mtr[0] + '.h5',
                                                           monitor=mtr[1],
                                                           save_best_only=True,
                                                           mode=mtr[2],
                                                           save_weights_only=True) for mtr in METRICS_CHECKPOINTS]

# Save training metrics evolution callback
metrics_evolution_callback = callbacks.Save_Training_Evolution(save_metrics_path + model_name_for_metrics + '_evolution.csv')


CBACKS = [learning_rate_scheduler_callback, checkpoint_callbacks, metrics_evolution_callback] 


# TRAIN
model.compile(optimizer=OPTIMIZER,
              loss=LOSS_FUNCTION,
              # SSIM_MS needs greater images than training patches, so ignore this metric on training
              metrics=[train.get_metric_function(x) for x in METRICS if x != ctes.METRIC_FUNCTIONS.SSIM_MS])

print('Starts training')
model.fit(train.dataset, epochs=NUM_EPOCHS, verbose=1, validation_data=val.dataset, callbacks=CBACKS)
print('Ends training')


with open('ended_scripts.txt', 'a') as f:
    f.write(FILENAME + '\n')

Starts training
Epoch 1/100
10000/10000 [==============================] - 6222s 622ms/step - loss: 6.0521 - psnr: 32.0584 - ssim: 0.8407 - mse: 138.2223 - mae: 5.5103 - sobel_loss: 2909.7810 - val_loss: 5.1866 - val_psnr: 30.2280 - val_ssim: 0.8599 - val_mse: 100.4500 - val_mae: 4.6600 - val_sobel_loss: 2190.8899
Epoch 2/100
10000/10000 [==============================] - 6181s 618ms/step - loss: 5.3709 - psnr: 33.2622 - ssim: 0.8618 - mse: 113.0640 - mae: 4.8426 - sobel_loss: 2437.6365 - val_loss: 4.9715 - val_psnr: 30.5703 - val_ssim: 0.8665 - val_mse: 93.5300 - val_mae: 4.4900 - val_sobel_loss: 2027.6069
Epoch 3/100
10000/10000 [==============================] - 6186s 619ms/step - loss: 5.1908 - psnr: 33.5848 - ssim: 0.8675 - mse: 106.5756 - mae: 4.6667 - sobel_loss: 2280.1675 - val_loss: 4.8683 - val_psnr: 30.7403 - val_ssim: 0.8699 - val_mse: 90.0800 - val_mae: 4.3900 - val_sobel_loss: 1944.4380
Epoch 4/100
10000/10000 [==============================] - 6190s 619ms/step - loss: 5.